In [364]:
import numpy as np
import pandas as pd
import random
random.uniform(0, 1)

0.3468446652009428

In [462]:
sequence=[0,1,2,3,4,3,2,1,0,4,3,2,1,2,3,4,3,2]
sequence=np.array(sequence)
intial=[0.05,0.5,0.3,0.05,1]
intial=np.array(intial)

In [491]:
### .............................................................Implementation of hidden markov model.................................................................
## 3 Taks to train HMM
  ## 1) Forward Algorithm
  ## 2) Backward Algorithm
  ## 3) Forward Backward Algorithm

class HMM:
    def __init__(self,sequence,initial_matrix,number_states=5,number_output=5,n_epochs=50):
        self.seq=sequence
        self.tras=np.random.uniform(0,1,(number_states,number_states))
        #self.tras=np.zeros(shape=(number_states,number_states))
        self.emiss=np.random.uniform(0,1,(number_states,number_output))
        #self.emiss=np.zeros(shape=(number_states,number_output))
        
        self.iprob=initial_matrix
        
        self.states=number_states
        
        self.out=number_output
        
        self.epochs=n_epochs
        
        self.BDP=np.zeros(shape=(number_states,len(self.seq)))
        
        self.FDP=np.zeros(shape=(number_states,len(self.seq)))
        
        self.n_epochs=n_epochs
        
    def forward(self):                                                          
        nstates=self.states                                                       
        nlength=len(self.seq)                                                     
        maxi=0
        index=0                                                                     
        for j in range(nlength):
            for i in range(nstates):                         
                if j==0:
                    self.FDP[i][j]=self.iprob[i]*self.emiss[i][self.seq[j]]
                else:
                    for k in range(nstates):
                        self.FDP[i][j]+=(self.FDP[k][j-1]*self.tras[k][i]*self.emiss[i][self.seq[j]])
                    if j==nlength-1:
                        if(self.FDP[i][j]>=maxi):
                            maxi=self.FDP[i][j]
                            index=i
        tp=0.0
        for i in range(nstates):
            tp+=self.FDP[i][nlength-1]
        return tp,maxi,index
    
    
    def decoding(self):                                                           
        nstates=self.states                                                       
        nlength=len(self.seq)                                                     
        tp,maxi,index=self.forward()
        hstates=np.zeros(shape=(nlength))
        for j in range(nlength-1,-1,-1):
            hstates[j]=index
            if(j==0):
                break
            for i in range(nstates):
                self.FDP[index][j]==self.FDP[i][j-1]*self.tras[i][index]*self.emiss[index][self.seq[j]]
                index=i
                break
        return hstates
    
        
    def backward(self):
        nstates=self.states                                                 
        nlength=len(self.seq)                                                     
        for j in range(nlength-1,-1,-1):
            for i in range(nstates):
                if(j==nlength-1):
                    self.BDP[i][j]=1.0
                else:
                    for k in range(nstates):
                        self.BDP[i][j]+=(self.tras[i][k]*self.emiss[k][self.seq[j+1]]*self.BDP[k][j+1])
                   
                
    def not_quite(self,time,curr_state,next_state):
        #self.forward()
        #self.backward()
        
        nstates=self.states                                                 
        nlength=len(self.seq)
        deno=0.00000000001
        for i in range(nstates):
            deno=self.FDP[i][time]*self.BDP[i][time]
        numo=self.FDP[curr_state][time]*self.tras[curr_state][next_state]*self.emiss[next_state][self.seq[time+1]]*self.BDP[next_state][time+1]
        return numo/deno
    
    
    
    def corrected_transition(self):
        nstates=self.states                                                 
        nlength=len(self.seq)
        
        for i in  range(nstates):
            for j in range(nstates):
                numo=0
                deno=0.00000000001
                for time in range(nlength-1):
                    numo+=self.not_quite(time,i,j)
                    for state in range(nstates):
                        deno+=self.not_quite(time,i,state)
                self.tras[i][j]=numo/deno
    
    
    def not_quite2(self,time,state):
        nstates=self.states                                                 
        nlength=len(self.seq)
        deno=0.00000000001
        for ti in range(nstates):
            deno+=self.FDP[state][ti]*self.BDP[state][ti]
        return self.FDP[state][time]*self.BDP[state][time]/deno
    
    
    def corrected_emmision(self):
        nstates=self.states                                                 
        nlength=len(self.seq)
        nout=self.out
        for states in (range(nstates)):
            for out in (range(nout)):
                numo=0
                deno=0.00000000001
                for time in range(nlength):
                    if(self.seq[time]==out):
                        numo+=self.not_quite2(time,states)
                    deno+=self.not_quite2(time,states)
                self.emiss[states][out]=numo/deno
            
            
    def train(self):
        for epoch in range(self.n_epochs):
            print("Epoc no : {} Traning".format(epoch))
            self.forward()
            self.backward()
            self.corrected_transition()
            self.corrected_emmision()
            print("Transition Matrix")
            print(self.tras)
            print("Emmission Matrix")
            print(self.emiss)

In [492]:
# deno+=self.not_quite(time,i,states)

In [493]:
hm=HMM(sequence,intial)

In [494]:
hm.train()

Epoc no : 0 Traning
Transition Matrix
[[0.3728767  0.33680136 0.21691141 0.17142856 0.25284134]
 [0.23201574 0.00589585 0.0702275  0.39532381 0.65887524]
 [0.18929294 0.11334178 0.07060046 0.27883752 0.68469697]
 [0.36114297 0.218411   0.33877167 0.03238399 0.36067872]
 [0.22646065 0.29169317 0.07225965 0.16631573 0.56175767]]
Emmission Matrix
[[0.02650565 0.07182246 0.47641198 0.25282905 0.17243086]
 [0.24384965 0.12656795 0.01086016 0.51812572 0.10059652]
 [0.18418988 0.17897628 0.20108045 0.39432794 0.04142545]
 [0.08153574 0.20005885 0.24271026 0.1968639  0.27883125]
 [0.04944027 0.26106042 0.39493615 0.07541245 0.21915071]]
Epoc no : 1 Traning
Transition Matrix
[[0.3553676  0.15298579 0.14907821 0.14719865 0.27739214]
 [0.18213048 0.00157255 0.03069223 0.27652369 0.60836521]
 [0.16099626 0.03051096 0.0353212  0.1985065  0.66701415]
 [0.34337889 0.11346314 0.22504977 0.02762832 0.36474224]
 [0.18055062 0.25483366 0.06074875 0.11571845 0.46074664]]
Emmission Matrix
[[0.02052532 0.07

Transition Matrix
[[2.22131320e-06 1.35119035e-12 1.59384713e-10 2.53757443e-07
  9.99999248e-01]
 [1.46115580e-07 8.47370364e-18 9.64397077e-12 8.38468574e-06
  9.99997056e-01]
 [3.10449456e-05 1.08428686e-10 2.09244902e-06 4.57051195e-02
  9.52463733e-01]
 [6.77765050e-06 2.15880857e-09 2.77623386e-07 1.86835445e-05
  9.99985249e-01]
 [9.85318038e-03 3.51624913e-01 6.25277508e-02 2.94788794e-02
  5.48896408e-01]]
Emmission Matrix
[[4.64353004e-02 1.72318500e-01 1.72267696e-01 5.92927312e-01
  1.60511908e-02]
 [1.15946040e-01 1.67353927e-01 2.05431018e-04 7.16008562e-01
  4.86040356e-04]
 [1.40450823e-01 2.83906884e-01 1.55779950e-02 5.59159878e-01
  9.04420514e-04]
 [1.82474730e-01 4.18316553e-01 7.17287983e-02 3.03869716e-01
  2.36102031e-02]
 [1.38379008e-02 1.69923160e-01 5.01689271e-01 2.12087235e-02
  2.93340945e-01]]
Epoc no : 20 Traning
Transition Matrix
[[6.80537421e-07 1.86620265e-13 2.66752634e-11 6.26135778e-08
  9.99999776e-01]
 [4.31398824e-08 8.48094171e-19 1.66975352e-

Transition Matrix
[[1.40954847e-13 7.11613271e-31 3.09491056e-28 1.61060260e-22
  1.00000000e+00]
 [6.44647364e-15 1.97668083e-41 5.66209103e-31 4.94558909e-20
  1.00000000e+00]
 [4.56723555e-11 1.21818021e-24 1.62274794e-16 2.83802482e-09
  9.99999999e-01]
 [1.05663911e-12 3.62308501e-23 2.51904314e-18 7.00068119e-14
  1.00000000e+00]
 [1.23186585e-02 3.09068103e-01 2.55308809e-02 3.85994985e-03
  6.50724450e-01]]
Emmission Matrix
[[1.13073762e-01 9.43550076e-02 1.50615653e-01 6.41096657e-01
  8.58920944e-04]
 [5.09787636e-01 7.11210139e-02 2.86168469e-05 4.19056558e-01
  6.17502005e-06]
 [5.21175177e-01 1.48220194e-01 2.75591832e-03 3.27835019e-01
  1.36918902e-05]
 [5.83745109e-01 2.11361928e-01 2.39282157e-02 1.80530292e-01
  4.34454668e-04]
 [7.11856648e-02 3.63121725e-01 3.21890103e-01 4.12884422e-03
  2.39673663e-01]]
Epoc no : 37 Traning
Transition Matrix
[[5.46930365e-14 3.17415040e-32 1.45693748e-29 1.35359258e-23
  1.00000000e+00]
 [2.45225774e-15 2.46600010e-43 1.56951857e-